In [1]:
import pandas as pd
import numpy as np


In [ ]:
counts_sex_age_state = pd.read_csv('cdc_death_counts_by_sex_age_state.csv')
counts_sex_age_state.head()

In [2]:
from sklearn.preprocessing import OneHotEncoder
def onehotencoder(data, column_name): 
    """
    One hot encodes a categorical feature. 
    Input:
      dataframe
      column_name with column_name to be one-hot-encoded 
    Output:
      A dataframe, with the one hot encoded features as columns.
    """
    column_data = data[[column_name]]
    ohe = OneHotEncoder()
    enc = ohe.fit_transform(column_data)
    enc_array = enc.toarray()
    one_hot_encoded_df = pd.DataFrame(data=enc_array, columns=ohe.get_feature_names_out\
                                      (input_features=[column_name]), index=data.index)
    data = data.drop(columns=[column_name])
    data = pd.concat([data, one_hot_encoded_df], axis=1)

    return data


jan - winter
feb - winter
march - spring
april- spring
may - spring
june - summer
july- summer
aug -summer
sept -fall
oct -fall
nov - fall
dec - winter

radomly pick 3 csvs from each month and group by season 

In [3]:
import os
import random
import pandas as pd

# Define the seasons
months = {
    '01': 'Jan', '02': 'Feb', '03': 'Mar', '04': 'Apr',
    '05': 'May', '06': 'Jun', '07': 'Jul', '08': 'Aug',
    '09': 'Sep', '10': 'Oct', '11': 'Nov', '12': 'Dec'
}

# Set the directory path where the CSVs are stored
directory = './datasets/csse_covid_19_daily_reports_us'

selected_csvs_by_month = {month: [] for month in months.keys()}

# List all files in the directory
files = os.listdir(directory)

# Iterate through files and group them by month
for file in files:
    for month in months.keys():
        if month in file.lower() and file.endswith('.csv'):
            selected_csvs_by_month[month].append(file)

# Load the selected CSVs into dataframes
dataframes_by_month = {month: [] for month in months.values()}

for month, csvs in selected_csvs_by_month.items():
    for csv in csvs:
        # Construct the file path
        file_path = os.path.join(directory, csv)

        # Load CSV into a dataframe
        df = pd.read_csv(file_path)

        # Append dataframe to the corresponding month
        dataframes_by_month[months[month]].append(df)

# Debugging statement
print(f'Number of dataframes for Jul: {len(dataframes_by_month["Jul"])}')

# if len(dataframes_by_month['Oct']) > 0:
#     dataframes_by_month['Oct'][0]
# else:
#     print("No dataframes available for 'fall' season.")

# dataframes_by_month['Jun'][10]
drop_states = ['American Samoa', 'Diamond Princess', 'Northern Mariana Islands'\
               , 'Puerto Rico', 'Virgin Islands', 'Grand Princess']
for month, dataframes in dataframes_by_month.items():
    modified_dataframes = []
    for df in dataframes:
        modified_df = df[~df['Province_State'].isin(drop_states)]
        modified_dataframes.append(modified_df)
    dataframes_by_month[month] = modified_dataframes
# dataframes_by_month['Oct'][11]

Number of dataframes for Jul: 83


In [4]:
data = {month: [df.drop(columns=['Last_Update', 'Country_Region', 'UID', 'ISO3', 'Lat', 'Long_',\
                                 'Recovered', 'Active', 'People_Hospitalized',\
                                   'FIPS', 'Hospitalization_Rate'])\
                for df in dataframes] for month, dataframes in dataframes_by_month.items()}


data = {month: [df.assign(Month=month) for df in dataframes] 
        for month, dataframes in data.items()}

data['Oct'][1]

,Province_State,Confirmed,Deaths,Incident_Rate,Total_Test_Results,Case_Fatality_Ratio,Testing_Rate,Month
0,Alabama,798320,14393,16281.661818,5758681.0,1.802911,117447.760996,Oct
1,Alaska,115703,578,15816.251905,3058523.0,0.499555,418090.889829,Oct
3,Arizona,1097225,20039,15074.428639,12756086.0,1.826335,175251.847269,Oct
4,Arkansas,496934,7718,16466.742042,3910410.0,1.553124,129577.997776,Oct
5,California,4746059,68841,12012.454981,92523189.0,1.450408,234163.461266,Oct
6,Colorado,673517,7594,11695.570000,11132130.0,1.127514,193308.566324,Oct
7,Connecticut,391066,8650,10968.710233,10027621.0,2.211903,281257.048872,Oct
8,Delaware,133599,1954,13719.854092,2213944.0,1.462586,227359.401251,Oct
10,District of Columbia,61331,1176,8690.200057,2044594.0,1.917464,289705.546873,Oct
11,Florida,3620106,55299,16855.155643,39774998.0,1.527552,185191.754606,Oct


In [7]:
concats = [pd.concat(df, axis=0) for df in data.values()]

all_data = pd.concat(concats, axis=0)


In [8]:
all_data = onehotencoder(all_data, 'Month')
all_data = onehotencoder(all_data, 'Province_State')
all_data.columns.values


array(['Confirmed', 'Deaths', 'Incident_Rate', 'Total_Test_Results',
       'Case_Fatality_Ratio', 'Testing_Rate', 'People_Tested',
       'Mortality_Rate', 'Month_Apr', 'Month_Aug', 'Month_Dec',
       'Month_Feb', 'Month_Jan', 'Month_Jul', 'Month_Jun', 'Month_Mar',
       'Month_May', 'Month_Nov', 'Month_Oct', 'Month_Sep',
       'Province_State_Alabama', 'Province_State_Alaska',
       'Province_State_Arizona', 'Province_State_Arkansas',
       'Province_State_California', 'Province_State_Colorado',
       'Province_State_Connecticut', 'Province_State_Delaware',
       'Province_State_District of Columbia', 'Province_State_Florida',
       'Province_State_Georgia', 'Province_State_Guam',
       'Province_State_Hawaii', 'Province_State_Idaho',
       'Province_State_Illinois', 'Province_State_Indiana',
       'Province_State_Iowa', 'Province_State_Kansas',
       'Province_State_Kentucky', 'Province_State_Louisiana',
       'Province_State_Maine', 'Province_State_Maryland',
       'P

In [49]:
print(all_data.dtypes)
all_data = all_data.astype(float)
nan_check = all_data.isna().any()

# Display columns with NaN values
columns_with_nan = nan_check[nan_check].index
print("Columns with NaN values:", columns_with_nan)
print(all_data.isna().sum())

Confirmed                       float64
Deaths                          float64
Incident_Rate                   float64
Total_Test_Results              float64
Case_Fatality_Ratio             float64
                                 ...   
Province_State_Virginia         float64
Province_State_Washington       float64
Province_State_West Virginia    float64
Province_State_Wisconsin        float64
Province_State_Wyoming          float64
Length: 73, dtype: object
Columns with NaN values: Index(['Incident_Rate', 'Total_Test_Results', 'Case_Fatality_Ratio',
       'Testing_Rate', 'People_Tested', 'Mortality_Rate'],
      dtype='object')
Confirmed                           0
Deaths                              0
Incident_Rate                      37
Total_Test_Results              25851
Case_Fatality_Ratio             25517
                                ...  
Province_State_Virginia             0
Province_State_Washington           0
Province_State_West Virginia        0
Province_State_Wi

In [72]:

# special_representations = ['NA', 'N/A', 'null']
# all_data.replace(special_representations, pd.NA, inplace=True)

# print(X['Deaths'].dtype)
import copy
X = copy.deepcopy(all_data).astype(float)

# X.dropna(subset=[col for col in X.columns if np.nan in X[col].values], inplace=True)
X = X.dropna(axis=1)
X
print(X.isna().sum())
target = X['Deaths']

# X_vals.drop(columns=['Deaths'], inplace=True)
X.shape

Confirmed                       0
Deaths                          0
Month_Apr                       0
Month_Aug                       0
Month_Dec                       0
                               ..
Province_State_Virginia         0
Province_State_Washington       0
Province_State_West Virginia    0
Province_State_Wisconsin        0
Province_State_Wyoming          0
Length: 67, dtype: int64


(83705, 67)

In [73]:
from sklearn.linear_model import LinearRegression
model = LinearRegression(fit_intercept=True)

In [74]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.2, random_state=42) 


In [82]:
model.fit(X_train, y_train) 

LinearRegression()

In [83]:
y_pred = model.predict(X_test)
y_pred

array([12514.,  7134.,  2038., ...,  3754.,  4084.,  9556.])

In [79]:
def rmse(predicted, actual):
    """
    Calculates RMSE from actual and predicted values.
    Input:
      predicted (1D array): Vector of predicted/fitted values
      actual (1D array): Vector of actual values
    Output:
      A float, the RMSE value.
    """
    return np.sqrt(np.mean((actual - predicted)**2))

In [92]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
model = LinearRegression(fit_intercept=True)
kf = KFold(n_splits=5, shuffle=True, random_state=42)  # Adjust the number of splits as needed

# List to store RMSE values for each fold
rmse_scores = []
fold = 0 
for train_index, test_index in kf.split(X):
    fold+=1
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]

    model.fit(X_train, y_train)

    predictions = model.predict(X_test)

    rmse_value = rmse(predictions, y_test)
    rmse_scores.append(rmse_value)
    print(f"Fold {fold} rmse: {rmse_value}")

average_rmse = np.mean(rmse_scores)
average_rmse

Fold 1 rmse: 1.7715756211162936e-11
Fold 2 rmse: 1.1349546068266364e-11
Fold 3 rmse: 2.3664979010096076e-11
Fold 4 rmse: 1.8516426541612768e-11
Fold 5 rmse: 1.9211853940532443e-11


1.8091712354334113e-11

In [86]:
rmse_err = rmse(y_pred, y_test)
rmse_err

1.4980689039553958e-11

In [87]:
# model_1 = LinearRegression(fit_intercept=True)